In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the rea"d-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-to-mongo/Train_NLQ_Mongo.csv
/kaggle/input/text-to-mongo/Test_NLQ_Mongo.csv
/kaggle/input/text-to-mongo/Val_NLQ_Mongo.json
/kaggle/input/text-to-mongo/new.csv
/kaggle/input/text-to-mongo/Val_NLQ_Mongo.csv
/kaggle/input/text-to-mongo/new.json
/kaggle/input/text-to-mongo/Train_NLQ_Mongo.json
/kaggle/input/text-to-mongo/Test_NLQ_Mongo.json


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [4]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset('csv', 
                       data_files={'train': '/kaggle/input/text-to-mongo/Train_NLQ_Mongo.csv',
                       'test' : '/kaggle/input/text-to-mongo/Test_NLQ_Mongo.csv',
                       'val' : '/kaggle/input/text-to-mongo/Val_NLQ_Mongo.csv',})

# new_data = load_dataset('csv', data_files= {'new' : '/kaggle/input/text-to-mongo/new.csv'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3ca6b13fed0a8be4/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
concatenated_dataset = concatenate_datasets([dataset['train'], dataset['test'],dataset['val']])

In [6]:
def get_training_corpus():
    query = []
    count = 0 
    for start_idx in range(0,( len(concatenated_dataset) ) ):
            
        row = concatenated_dataset[start_idx]

        query.append(row['NLQ'])
        query.append(row['Mongo'])
        count += 1 

        if count == 1000:    
            yield np.array(query)
            count = 0
            query = []

training_corpus = get_training_corpus()
training_corpus

<generator object get_training_corpus at 0x7db44ad45c40>

In [7]:
tokenizer = tokenizer.train_new_from_iterator(training_corpus,vocab_size = 20000)

In [8]:
tokenizer.save_pretrained("./output/Tokenizer/Final with bleu Tokenized Version Tokenizer.h5")

('./output/Tokenizer/Final with bleu Tokenized Version Tokenizer.h5/tokenizer_config.json',
 './output/Tokenizer/Final with bleu Tokenized Version Tokenizer.h5/special_tokens_map.json',
 './output/Tokenizer/Final with bleu Tokenized Version Tokenizer.h5/tokenizer.json')

In [ ]:
train = dataset['train']
val = dataset['val']

def preprocess_function(examples):
    inputs = examples["NLQ"]
    targets = examples["Mongo"]
    model_inputs = tokenizer(
        inputs, text_target=targets, 
#         padding='max_length', 
        truncation=True
    )
    return model_inputs

train_tokenized_datasets = train.map(
    preprocess_function,
    batched=True,
    remove_columns=train.column_names,
)

val_tokenized_datasets = val.map(
    preprocess_function,
    batched=True,
    remove_columns=train.column_names,
)

In [ ]:
# from transformers import Trainer, 
from transformers import DataCollatorForSeq2Seq

In [ ]:
import evaluate

bleu = evaluate.load("bleu")

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result}

In [4]:
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    save_steps=1750,
    save_total_limit=2,
    learning_rate=0.001,
    weight_decay=0,
    logging_strategy = 'steps',
    logging_steps=250,
    evaluation_strategy='steps',
    eval_steps=1000,
    fp16=True,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model = model,
    tokenizer = tokenizer,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=val_tokenized_datasets,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )

KeyboardInterrupt: 

In [ ]:
trainer.train()

In [ ]:
result = trainer.evaluate()

In [ ]:
print('bleu : ',result['eval_bleu'])

In [ ]:
model.num_parameters() / 1_000_000

In [ ]:
from matplotlib import pyplot as plt

def plot(path,data):
    
    loss = []
    rate = []
    step = []
    eval_step = []
    vline = []
    eval_loss = []
    bleu = []
    precisions = [[],[],[],[]]

    for i in range(0,len(data)): 
        
        try:
            loss.append(data[i]['loss'])
            rate.append(data[i]['learning_rate'])
            step.append(data[i]['step'])
        except:
            eval_loss.append(data[i]['eval_loss'])
            eval_step.append(data[i]['step'])
            bleu.append(data[i]['eval_bleu']['bleu'])
            precisions[0].append(data[i]['eval_bleu']['precisions'][0])
            precisions[1].append(data[i]['eval_bleu']['precisions'][1])
            precisions[2].append(data[i]['eval_bleu']['precisions'][2])
            precisions[3].append(data[i]['eval_bleu']['precisions'][3])
        

#         if data[i]['epoch'] == round(data[i]['epoch']) or data[i]['step']:
#             vline.append(data[i]['step'])

    plt.plot(step, loss, marker='o', linestyle='-', color='b',label= 'Train Loss')
    plt.plot(eval_step, eval_loss, marker='o', linestyle='-', color='r', label= 'Eval Loss')
    plt.legend()
    
#     for each in vline:
#         plt.axvline(x=each, color='gray', linestyle='--')

    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.title('Training Loss vs. Step')
    plt.savefig(path + 'Loss.png')
    plt.show()

    plt.plot(step, rate, marker='o', linestyle='-', color='g')

#     for each in vline:
#         plt.axvline(x=each, color='gray', linestyle='--')

    plt.xlabel('Step')
    plt.ylabel('Learning Rate')
    plt.title('Learning Rate vs. Step')
    plt.savefig(path + 'Learning Rate.png')
    plt.show()
    
    plt.plot([0] + eval_step, [ 0.013] + bleu, marker='o', linestyle='-', color='g')

#     for each in vline:
#         plt.axvline(x=each, color='gray', linestyle='--')

    plt.xlabel('Step')
    plt.ylabel('BLEU Score')
    plt.title('Bleu Score vs. Step')
    plt.savefig(path + 'Bleu score.png')
    plt.show()
    
    plt.plot([0] + eval_step, [0.197] +precisions[0], marker='o', linestyle='-', color='g',
            label = '1-gram')
    plt.plot([0] + eval_step, [ 0.066] +precisions[1], marker='o', linestyle='-', color='r',
            label = '2-gram')
    plt.plot([0] + eval_step, [ 0.028] +precisions[2], marker='o', linestyle='-', color='b',
            label = '3-gram')
    plt.plot([0] + eval_step, [ 0.014] +precisions[3], marker='o', linestyle='-', color='#00FF66',
            label = '4-gram')

#     for each in vline:
#         plt.axvline(x=each, color='gray', linestyle='--')
    plt.legend()
    plt.xlabel('Step')
    plt.ylabel('BLEU N-gram precision')
    plt.title('Bleu Score vs. Step')
    plt.savefig(path + 'Bleu precisions.png')
    plt.show()
    

In [ ]:

data = trainer.state.log_history

plot('./output/',data[:-4])


In [ ]:
model.save_pretrained('./output/Models/Final with bleu Tokenized Version.h5')

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained('./output/Models/First Tokenized Version.h5')

NameError: name '_C' is not defined

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained('./output/Models/Final with bleu Tokenized Version.h5')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [5]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
# model_parameters = model.state_dict()
# for param_name, param_tensor in model_parameters.items():
# #     print(f"Parameter name: {param_name}, Shape: {param_tensor.shape}")

In [7]:
!pip install torchview

In [11]:
from torchview import draw_graph
input_text = ["what are all the result for New York 6 district"]
inputs = tokenizer(input_text, return_tensors="pt")

model_graph = draw_graph(model, input_data=inputs).visual_graph
# Line below triggers an error: "RuntimeError: Failed to run torchgraph"
model_graph.render(directory='/home/jeff/tmp/graphviz_output').replace('\\', '/')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/torchview/torchview.py:258 in forward_prop               │
│                                                                                                  │
│   255 │   │   │   │   if isinstance(x, (list, tuple)):                                           │
│   256 │   │   │   │   │   _ = model.to(device)(*x, **kwargs)                                     │
│   257 │   │   │   │   elif isinstance(x, Mapping):                                               │
│ ❱ 258 │   │   │   │   │   _ = model.to(device)(**x, **kwargs)                                    │
│   259 │   │   │   │   else:                                                                      │
│   260 │   │   │   │   │   # Should not reach this point, since process_input_data ensures        │
│   261 │   │   │   │   │   # x is either a list, tuple, or Mapping                                │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torchview/recorder_tensor.py:146 in                      │
│ _module_forward_wrapper                                                                          │
│                                                                                                  │
│   143 │   │                                                                                      │
│   144 │   │   # TODO: check if output contains RecorderTensor                                    │
│   145 │   │   # this seems not to be necessary so far                                            │
│ ❱ 146 │   │   out = _orig_module_forward(mod, *args, **kwargs)                                   │
│   147 │   │                                                                                      │
│   148 │   │   model_graph.context_tracker['current_depth'] = cur_depth                           │
│   149 │   │   model_graph.context_tracker['current_context'] = input_context                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/models/t5/modeling_t5.py:1720 in forward    │
│                                                                                                  │
│   1717 │   │   │   │   decoder_attention_mask = decoder_attention_mask.to(self.decoder.first_de  │
│   1718 │   │                                                                                     │
│   1719 │   │   # Decode                                                                          │
│ ❱ 1720 │   │   decoder_outputs = self.decoder(                                                   │
│   1721 │   │   │   input_ids=decoder_input_ids,                                                  │
│   1722 │   │   │   attention_mask=decoder_attention_mask,  

In [9]:

input_text = ["what are all the result for New York 6 district"]

input_ids = tokenizer(input_text, return_tensors="pt").input_ids
inputs = tokenizer(input_text, return_tensors="pt")

output = model.generate(input_ids)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)



/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


hal– pre said pre He


In [23]:
input_text2 = ["What does kaustub eat?"]
input_ids = tokenizer(input_text2, return_tensors="pt").input_ids

output = model.generate(input_ids)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

db.table.find({"Name": "kaustub eat"}, {"Kitub": 1})


In [24]:
input_text2 = ["What is terrence ross' nationality?"]
input_ids = tokenizer(input_text2, return_tensors="pt").input_ids

output = model.generate(input_ids)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

db.table.find({"Player": "terrence ross"}, {"Nationality": 1})


In [25]:
input_text2 = ["Which class does kaustub belong to?"]
input_ids = tokenizer(input_text2, return_tensors="pt").input_ids

output = model.generate(input_ids)

output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

db.table.find({"Player": "kaustub"}, {"Class": 1})


In [ ]:
# def get_training_corpus():
#     query = []
#     count = 0 
#     for start_idx in range(0,( len(concatenated_dataset) + len(new_data['new'])) ):
        
#         if start_idx < len(new_data['new']):
           
#             query.append(new_data['new']['1'][start_idx])
            
#             count += 1
        
#             if count == 1000:    
#                 yield np.array(query)
#                 count = 0
#                 query = []
                
#         else:
#             new_idx = start_idx - len(new_data['new'])
            
#             row = concatenated_dataset[new_idx]

#             query.append(row['NLQ'])
#             query.append(row['Mongo'])
#             count += 1 
            
#             if count == 1000:    
#                 yield np.array(query)
#                 count = 0
#                 query = []

# training_corpus = get_training_corpus()
# training_corpus